# Агенство недвижимости

**Цели:**  
Выявить наиболее привлекательные сегменты рынка по времени активности объявлений.  
Проанализировать сезонность публикации и снятия объявлений.  
Сформировать выводы и практические рекомендации для агентства недвижимости.

## Содержимое проекта  
   1. [Описание данных](#1-bullet)
   2. [Подключение к БД](#2-bullet)
   3. [Анализ данных](#3-bullet)
   4. [Общие выводы и рекомендации](#4-bullet)

<a class="anchor" id="1-bullet"></a>
## Описание данных

4 таблицы:

**advertisement** - содержит информацию об объявлениях  
id — идентификатор объявления (первичный ключ).  
first_day_exposition — дата подачи объявления.  
days_exposition — длительность нахождения объявления на сайте (в днях).  
last_price — стоимость квартиры в объявлении, в руб.  

**flats** - содержит информацию о квартирах  
id — идентификатор квартиры (первичный ключ, связан с первичным ключом id таблицы advertisement).  
city_id — идентификатор города (внешний ключ, связан с city_id таблицы city).   
type_id — идентификатор типа населённого пункта (внешний ключ, связан с type_id таблицы type).  
total_area — общая площадь квартиры, в кв. метрах.  
rooms — число комнат.  
ceiling_height — высота потолка, в метрах.  
floors_total — этажность дома, в котором находится квартира.  
living_area — жилая площадь, в кв. метрах.  
floor — этаж квартиры.  
is_apartment — указатель, является ли квартира апартаментами (1 — является, 0 — не является).  
open_plan — указатель, имеется ли в квартире открытая планировка (1 — открытая планировка квартиры, 0 — открытая планировка отсутствует).  
kitchen_area — площадь кухни, в кв. метрах.  
balcony — количество балконов в квартире.  
airports_nearest — расстояние до ближайшего аэропорта, в метрах.  
parks_around3000 — число парков в радиусе трёх километров.  
ponds_around3000 — число водоёмов в радиусе трёх километров.  

**city** - содержит информацию о городах  
city_id — идентификатор населённого пункта (первичный ключ).  
city — название населённого пункта.  

**type** - содержит информацию о городах  
type_id — идентификатор типа населённого пункта (первичный ключ).  
type — название типа населённого пункта.  

![Example Image](image_1738083790.png)

<a class="anchor" id="2-bullet"></a>
## Подключение к БД

Используем подключение через SQLAlchemy, потому что Pandas официально поддерживает только три типа соединений для pd.read_sql(), и прямое подключение через psycopg2 не входит в их число.  
Подключение через SQLAlchemy дает:  
    - автоматически управляет соединением (открывает/закрывает)  
    - поддерживает разные СУБД без изменения кода  
    - удобен для сложных запросов и транзакций

In [1]:
pip install sqlalchemy

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [2]:
# Библиотеки
from sqlalchemy import create_engine
import pandas as pd

In [3]:
# Подключение к БД

# Параметры подключения
username = '**'
password = '**'
host = '**'
port = '**'
database = '**'

# Формируем строку подключения
engine = create_engine(f'postgresql+psycopg2://{username}:{password}@{host}:{port}/{database}')

<a class="anchor" id="3-bullet"></a>
## Анализ данных

### Время активности объявлений

In [4]:
task_1 = """
-- Задача 1: Время активности объявлений
-- Определим аномальные значения (выбросы) по значению перцентилей:
WITH limits AS (
    SELECT
        PERCENTILE_CONT(0.99) WITHIN GROUP (ORDER BY total_area) AS total_area_limit,
        PERCENTILE_DISC(0.99) WITHIN GROUP (ORDER BY rooms) AS rooms_limit,
        PERCENTILE_DISC(0.99) WITHIN GROUP (ORDER BY balcony) AS balcony_limit,
        PERCENTILE_CONT(0.99) WITHIN GROUP (ORDER BY ceiling_height) AS ceiling_height_limit_h,
        PERCENTILE_CONT(0.01) WITHIN GROUP (ORDER BY ceiling_height) AS ceiling_height_limit_l
    FROM real_estate.flats
),
-- Найдём id объявлений, которые не содержат выбросы, также оставим пропущенные данные:
filtered_id AS(
    SELECT id
    FROM real_estate.flats
    WHERE
        total_area < (SELECT total_area_limit FROM limits)
        AND (rooms < (SELECT rooms_limit FROM limits) OR rooms IS NULL)
        AND (balcony < (SELECT balcony_limit FROM limits) OR balcony IS NULL)
        AND ((ceiling_height < (SELECT ceiling_height_limit_h FROM limits)
            AND ceiling_height > (SELECT ceiling_height_limit_l FROM limits)) OR ceiling_height IS NULL)
    ), 
-- Разделим на категории по кол-ву дней существования объявления
category_days AS(
	SELECT *, 
			CASE WHEN days_exposition<=30 AND days_exposition>=1 THEN '1-30 days'
					WHEN days_exposition<=90 AND days_exposition>=31 THEN '31-90 days'
					WHEN days_exposition<=180 AND days_exposition>=91 THEN '91-180 days'
					WHEN days_exposition<=1580 AND days_exposition>=181 THEN '181+ days'
					ELSE 'non category'
			END AS category_days
	FROM real_estate.advertisement
),
--Категория по региону
category_city AS(
	SELECT *, 
			CASE WHEN city = 'Санкт-Петербург' THEN 'Санкт-Петербург' ELSE 'Ленинградская область' END AS region
	FROM real_estate.city 
)
--Основной запрос
SELECT region, 
		category_days , 
		COUNT(id) AS count_advertisement,
		ROUND(AVG(last_price / total_area)::NUMERIC ,2) AS avg_price_one_meter, 
		ROUND(AVG(total_area)::NUMERIC,2) AS avg_total_area,
		ROUND(AVG(last_price)::NUMERIC,2) AS avg_price,
		PERCENTILE_CONT(0.5) WITHIN GROUP (ORDER BY rooms) AS median_rooms,
		PERCENTILE_CONT(0.5) WITHIN GROUP (ORDER BY balcony ) AS median_balcony,
		PERCENTILE_CONT(0.5) WITHIN GROUP (ORDER BY floor ) AS median_floor,
		ROUND(100.0 * COUNT(*) / SUM(COUNT(*)) OVER (PARTITION BY region), 2) AS share_in_region_pct,  -- Доля категории от всех объявлений в этом регионе
		ROUND(AVG(ceiling_height)::NUMERIC,2) AS avg_ceiling_height,  -- Ср высота потолка
		ROUND(AVG(airports_nearest)::NUMERIC,2) AS avg_airports_nearest, -- Ср расстояние до аэропорта
		PERCENTILE_CONT(0.5) WITHIN GROUP (ORDER BY parks_around3000) AS median_parks_around, -- Медиана парков
		PERCENTILE_CONT(0.5) WITHIN GROUP (ORDER BY ponds_around3000) AS median_ponds_around, -- Медиана водоемов
		ROUND(100.0 * SUM(CASE WHEN rooms = 0 THEN 1 ELSE 0 END) / COUNT(*)::NUMERIC, 2) AS studios_pct, -- Процент студий
		ROUND(100.0 * SUM(CASE WHEN is_apartment = 1 THEN 1 ELSE 0 END) / COUNT(*)::NUMERIC, 2) AS apartment_pct, -- Процент апартаментов
		ROUND(100.0 * SUM(CASE WHEN open_plan = 1 THEN 1 ELSE 0 END) / COUNT(*)::NUMERIC, 2) AS open_plan_pct -- Процент свободной планировки
FROM real_estate.flats
LEFT JOIN category_days USING(id)
LEFT JOIN category_city USING(city_id )
LEFT JOIN real_estate.type  USING(type_id)
--Выделим только нужные id, тип город и даты с 2015 по 2018 год
WHERE id IN (SELECT * FROM filtered_id) AND (TYPE = 'город') AND (EXTRACT(YEAR FROM first_day_exposition) >= '2015' AND EXTRACT(YEAR FROM first_day_exposition) <= '2018')
GROUP BY region, category_days; 
"""

In [5]:
df_1 = pd.read_sql(task_1, con=engine)
df_1

,region,category_days,count_advertisement,avg_price_one_meter,avg_total_area,avg_price,median_rooms,median_balcony,median_floor,share_in_region_pct,avg_ceiling_height,avg_airports_nearest,median_parks_around,median_ponds_around,studios_pct,apartment_pct,open_plan_pct
0,Ленинградская область,1-30 days,340,71907.63,48.75,3500117.40,2.0,1.0,4.0,12.02,2.70,32776.40,1.0,1.0,0.88,0.59,0.59
1,Ленинградская область,181+ days,873,68215.11,55.03,3773134.33,2.0,1.0,3.0,30.87,2.72,35284.23,0.0,1.0,0.11,0.11,0.00
2,Ленинградская область,31-90 days,864,67423.80,50.85,3417498.14,2.0,1.0,3.0,30.55,2.71,36789.42,0.0,1.0,0.58,0.12,0.23
3,Ленинградская область,91-180 days,553,69809.30,51.83,3620472.79,2.0,1.0,3.0,19.55,2.70,33480.62,1.0,1.0,0.90,0.00,0.18
4,Ленинградская область,non category,198,72925.89,62.78,4674793.98,2.0,1.0,3.0,7.00,2.80,31113.40,1.0,0.0,1.01,0.51,0.00
5,Санкт-Петербург,1-30 days,1794,108919.78,54.66,6092949.22,2.0,1.0,5.0,15.99,2.76,27514.68,0.0,0.5,1.56,0.22,0.28
6,Санкт-Петербург,181+ days,3506,114981.07,65.76,7980344.62,2.0,1.0,5.0,31.26,2.83,27394.34,0.0,1.0,0.48,0.14,0.11
7,Санкт-Петербург,31-90 days,3020,110874.32,56.58,6473286.13,2.0,1.0,5.0,26.92,2.77,28095.45,0.0,1.0,1.19,0.10,0.60
8,Санкт-Петербург,91-180 days,2244,111973.67,60.55,6998081.82,2.0,1.0,5.0,20.01,2.79,28134.48,0.0,1.0,0.58,0.18,0.18
9,Санкт-Петербург,non category,653,136107.66,81.38,11418045.70,3.0,1.0,4.0,5.82,2.90,27749.87,1.0,1.0,0.46,1.07,0.00


Выводы:

1. В обоих регионах превалируют категории со временем продажи около 3х месяцев (группы 31-90 и 181+ дней). Ждать быстрых сделок не стоит.  
2. Дорогие квартиры чаще висят на рынке дольше. Крупные квартиры дороже и продаются медленнее.  
Быстрее всего уходят недорогие квартиры в области. В Лен. области квартиры с ценой 67-72 тыс.руб/кв.м. (группы 31-90 и 1-30 дней)  
Быстрые сделки в Санкт-Петербурге приходятся на квартиры с ценой 109 тыс.руб/кв.м. (группы 31-90 и 1-30 дней)  
Дольше всего уходят дорогие квартиры в Санкт-Петербурге.  
Медианное кол-во комнат в областях одинаковое - 2 комнаты. Однако в Питере чаще встречаются 3х комнатные квартиры.  
Кол-во балконов почти не оказывает влияния. В среднем по обеим территориям ол-во балконов = 1.  
3. Различия между недвижимостью Санкт-Петербурга и Ленинградской области:  
В Санкт-Петербурге цены выше в 2 раза, И по общей стоимости и за кв.м.  
В Санкт-Петербурге больше больших квартир (до 81 кв.м.)  
В Санкт-Петербурге выше этажность - 5 этаж, в области - 3  
В области более равномерное распределение объявлений по категориям Дней жизни объявления. В Питере сильнее перевешивают долго не продающиеся. 

### Сезонность объявлений

In [6]:
task_2 = """
-- Задача 2: Сезонность объявлений
-- Определим аномальные значения (выбросы) по значению перцентилей:
SET lc_time = 'ru_RU';
WITH limits AS (
    SELECT
        PERCENTILE_CONT(0.99) WITHIN GROUP (ORDER BY total_area) AS total_area_limit,
        PERCENTILE_DISC(0.99) WITHIN GROUP (ORDER BY rooms) AS rooms_limit,
        PERCENTILE_DISC(0.99) WITHIN GROUP (ORDER BY balcony) AS balcony_limit,
        PERCENTILE_CONT(0.99) WITHIN GROUP (ORDER BY ceiling_height) AS ceiling_height_limit_h,
        PERCENTILE_CONT(0.01) WITHIN GROUP (ORDER BY ceiling_height) AS ceiling_height_limit_l
    FROM real_estate.flats
),
-- Найдём id объявлений, которые не содержат выбросы, также оставим пропущенные данные:
filtered_id AS(
    SELECT id
    FROM real_estate.flats
    WHERE
        total_area < (SELECT total_area_limit FROM limits)
        AND (rooms < (SELECT rooms_limit FROM limits) OR rooms IS NULL)
        AND (balcony < (SELECT balcony_limit FROM limits) OR balcony IS NULL)
        AND ((ceiling_height < (SELECT ceiling_height_limit_h FROM limits)
            AND ceiling_height > (SELECT ceiling_height_limit_l FROM limits)) OR ceiling_height IS NULL)
    ),
--Сначала добавим день снятия с продажи и применим фильтры:
final_data AS(
	SELECT *, first_day_exposition + (days_exposition || ' days')::INTERVAL AS end_day_exposition
	FROM real_estate.flats
	LEFT JOIN real_estate.advertisement USING(id)
	LEFT JOIN real_estate.type USING(type_id)
	WHERE id IN (SELECT * FROM filtered_id) AND (TYPE = 'город') AND (EXTRACT(YEAR FROM first_day_exposition) >= '2015' AND EXTRACT(YEAR FROM first_day_exposition) <= '2018')
),
--Временной ряд по месяцам количества снятых/выставленных общявлений
new_adv AS (
SELECT
	EXTRACT(MONTH FROM first_day_exposition) AS month_num,
	TO_CHAR(first_day_exposition, 'TMmonth') AS month_name,
	COUNT(*) AS new_adv,
	ROUND(AVG(total_area)::NUMERIC , 2) AS avg_new_total_area,  -- Добавим метрики средняя стоимость за кв метр и средняя площадь квартиры
	ROUND(AVG(last_price / total_area::float)::NUMERIC , 2) AS avg_new_price_meter,
	RANK() OVER (ORDER BY COUNT(*) DESC) AS new_adv_rank, -- Добавили ранги
	ROUND(100.0 * COUNT(*) / SUM(COUNT(*)) OVER (), 2) AS new_adv_pct -- Процент опубликованных от общего
FROM final_data
GROUP BY month_num,
		 month_name
),
removed_adv AS (
SELECT
	EXTRACT(MONTH FROM end_day_exposition) AS month_num,
	TO_CHAR(end_day_exposition, 'TMmonth') AS month_name,
	COUNT(*)AS removed_adv,
	ROUND(AVG(total_area)::NUMERIC , 2) AS avg_removed_total_area,   --Добавим метрики средняя стоимость за кв метр и средняя площадь квартиры
	ROUND(AVG(last_price / total_area::float)::NUMERIC , 2) AS avg_removed_price_meter,
	RANK() OVER (ORDER BY COUNT(*) DESC) AS removed_adv_rank, -- Добавили ранги
	ROUND(100.0 * COUNT(*) / SUM(COUNT(*)) OVER (), 2) AS removed_adv_pct -- Процент опубликованных от общего
FROM final_data
GROUP BY month_num, 
		 month_name
)
SELECT month_name,
		new_adv,
		removed_adv,
		new_adv_rank,
		removed_adv_rank,
		new_adv_pct,
		removed_adv_pct,
		avg_new_total_area,
		avg_removed_total_area,
		avg_new_price_meter,
		avg_removed_price_meter
FROM new_adv
LEFT JOIN removed_adv USING(month_num, month_name)
ORDER BY month_num;
"""

In [7]:
df_2 = pd.read_sql(task_2, con=engine)
df_2

,month_name,new_adv,removed_adv,new_adv_rank,removed_adv_rank,new_adv_pct,removed_adv_pct,avg_new_total_area,avg_removed_total_area,avg_new_price_meter,avg_removed_price_meter
0,января,735,1225,12,4,5.23,8.72,59.16,57.53,106106.24,104947.31
1,февраля,1369,1048,3,9,9.75,7.46,60.10,61.12,103058.51,103883.72
2,марта,1119,1071,8,8,7.97,7.63,60.00,60.37,102429.95,106832.40
3,апреля,1021,1031,10,10,7.27,7.34,60.60,59.22,102632.41,102444.24
4,мая,891,729,11,13,6.34,5.19,59.19,57.78,102465.12,99724.07
5,июня,1224,771,5,12,8.71,5.49,58.37,59.82,104802.15,101863.69
6,июля,1149,1108,7,7,8.18,7.89,60.42,58.54,104488.96,102290.72
7,августа,1166,1137,6,6,8.30,8.10,58.99,56.83,107034.70,100036.51
8,сентября,1341,1238,4,3,9.55,8.81,61.04,57.49,107563.12,104070.07
9,октября,1437,1360,2,1,10.23,9.68,59.43,58.86,104065.11,104317.33


Выводы:

1. В какие месяцы наблюдается наибольшая активность в публикации объявлений о продаже недвижимости?  
Лидеры по публикациям - Сентябрь, Октябрь, Февраль, минималоьное количество - Май  
Лидеры по снятию - Сентябрь, Октябрь, Ноябрь, минимальное количество - Май  
Пик приходится на осень.  
2. Совпадают ли периоды активной публикации объявлений и периоды, когда происходит повышенная продажа недвижимости?  
Да, выявился сильный корреляционный эффект. Чем больше в месяц выложено новых предложений, тем больше покупок будет в этом месяце.
Значит рынок движется синхронно.
3. 
Значения средней стоимости кв. метра колеблются в диапазоне 102-107тыс.  
Самые дорогие  месяцы по публикациям: Август (107тыс), Сентябрь (107,5тыс)  
Самые дорогие  месяцы по снятию: Март (107тыс), Декабрь(105тыс)  
Самые большие по площади квартиры продаются в Феврале и Марте 60-61 кв.м  
Самые мелкие в Августе и Ноябре 56 кв.м  
На общем фоне сильнее всего по показателям отстает Май

<a class="anchor" id="4-bullet"></a>
## Общие выводы и рекомендации:

Быстрые сделки редки, особенно в Санкт-Петербурге. Они идут в течении 3 месяцев и более.  
Дорогие и крупные квартиры особенно долго продаются. Быстрее всего продать маленькие квартиры в области ценой 67-72 тыс.руб/кв.м. В Питере 109 тыс.руб/кв.м.  
В Санкт-Петербурге цены в 2 раза выше, квартиры большей площадью.  
Количество новых объявлений и снятых с публикации сильно коррелирует.  
Средняя площадь квартир уменьшается со временем.  
В среднем квартиры имеют 2 комнаты и 1 балкон.  
Пик активности публикаций и снятия приходится на осень. Минимальная активность - Май.

**Рекомендации:**  

Планировать продажи исходя из общих тенденций рынка.  
В Ленинградской области делать упор на небольшие и недорогие квартиры. В Санкт-Петербурге можно экспериментировать с большими квартирами, но не ждать быстрых продаж.  
Планировать активные маркетинговые компании на осень, в мае снизить.  
В рекламе делать упор на количество комнат и цену за кв.м. 